In [1]:
import numpy as np
from os.path import join

In [2]:
root = '../orange_demo/mv'

In [3]:
data = np.load(join(root,'dist.npy'), allow_pickle=True)

In [4]:
print(len(data[0][1][0]))
print(len(data[0][1]))
print(len(data[0]))
print(len(data))

3264
1
6
850


In [5]:
query_paths = []
with open(join(root,'queries_paths.txt'), 'r') as f:
    for path in f.readlines():
        query_paths.append(path.strip())

gallery_paths = []
with open(join(root,'galleries_paths.txt'), 'r') as f:
    for path in f.readlines():
        gallery_paths.append(path.strip())

In [6]:
print(len(query_paths))
print(query_paths[0])
print(len(gallery_paths))
print(gallery_paths[0])
print(gallery_paths[1])


850
0000_t0000123567
17242
0000_c1_t0020.png
0000_c1_t0030.png


In [7]:
q_pids = []
q_camids = []
for qpath in query_paths:
    q_pids.append(int(qpath[:4]))
    tmp_camids = []
    for c in qpath[10:]:
        tmp_camids.append(int(c))
    q_camids.append(tmp_camids)




In [8]:
from collections import defaultdict
g_pids = defaultdict(list)
g_paths = defaultdict(list)
for gpath in gallery_paths:
    camid = int(gpath[6])
    g_pids[camid].append(int(gpath[:4]))
    g_paths[camid].append(gpath)

In [9]:
g_pids

defaultdict(list,
            {1: [0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              1,
              2,
              2,
              3,
              3,
              3,
              3,
              3,
              4,
              4,
              4,
              4,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              6,
              7,
              8,
              8,
              8,
              10,
         

In [10]:
from tqdm import tqdm
def match(dist,k=5):
    output_match_dict = defaultdict(dict)
    for frame_idx, frame_batch in tqdm(enumerate(dist)):
        cur_batch_dict = defaultdict(list)
        for cam_idx, cam_dist in enumerate(frame_batch):
            num_q, num_g = cam_dist.shape
            # print('numq,numg: {},{}'.format(num_q,num_g))
            # 目前不需要判断
            # if num_g < max_rank:
            #     max_rank = num_g
            #     print(
            #         'Note: number of gallery samples is quite small, got {}'.format(num_g))
            # print('camidx,camdist: {},{}'.format(cam_idx,cam_dist))
            indice = np.argsort(cam_dist, axis=1)
            indice = np.squeeze(indice)
            # print('indice:{}'.format(indice))
            # print(type(indice),indice.shape)
            q_pid = q_pids[frame_idx]

            camid = q_camids[frame_idx][cam_idx]

            ordered_dist = cam_dist[0][indice]

            # print(g_pids[camid])
            # print(len(g_pids[camid]))
            # print(indice)
            # print(len(indice))
            arr_g_pids = np.array(g_pids[camid])
            predict = arr_g_pids[indice]
            # predict = g_pids[camid][indice]
            # print(4)
            match = (predict == q_pid)
            # print(match)
            # print(type(match))
            # print(len(match))
            # print(sum(match))
            curcam_tmp_match = []
            for i in range(k):
                curcam_tmp_match.append([g_paths[camid][indice[i]],str(match[i]),str(cam_dist[0][indice[i]])])
            cur_batch_dict[camid].append(curcam_tmp_match)
        # print(frame_idx)
        # print(query_paths[frame_idx])
        output_match_dict[query_paths[frame_idx][:10]] = cur_batch_dict
    return output_match_dict
res= match(data, 5)

850it [00:01, 618.87it/s]


In [11]:
res

defaultdict(dict,
            {'0000_t0000': defaultdict(list,
                         {1: [[['0001_c1_t0000.png', 'False', '0.24084616'],
                            ['0001_c1_t0005.png', 'False', '0.2867877'],
                            ['0000_c1_t0075.png', 'True', '0.34235716'],
                            ['0000_c1_t0080.png', 'True', '0.3500219'],
                            ['0001_c1_t0140.png', 'False', '0.3508383']]],
                          2: [[['0001_c2_t0000.png', 'False', '0.042776108'],
                            ['0001_c2_t0005.png', 'False', '0.15816104'],
                            ['0000_c2_t0050.png', 'True', '0.27324092'],
                            ['0000_c2_t0075.png', 'True', '0.31220865'],
                            ['0000_c2_t0080.png', 'True', '0.32626688']]],
                          3: [[['0000_c3_t0105.png', 'True', '0.34175026'],
                            ['0000_c3_t0040.png', 'True', '0.3806085'],
                            ['0000_c3_t0095.pn

In [12]:
import json
# 定义一个处理不可序列化对象的函数
# def convert_to_builtin_type(obj):
#     if isinstance(obj, bool):
#         return str(obj)  # 或者 return int(obj)
#     raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

# # 使用 convert_to_builtin_type 函数进行 JSON 序列化
# json_str = json.dumps(res, default=convert_to_builtin_type)

# 将 JSON 字符串写入文件
with open(join(root, 'matches.json'), 'w') as f:
    json.dump(res, f, indent=4, sort_keys=True)

In [23]:
0 != bool('True')

True

In [25]:
bool('0')

True

In [3]:
with open('a.txt', 'w') as f:
    f.write('hi')